# Simulating genomic data in a 'plant breeding' context

In [1]:
using XSim, DelimitedFiles, Distributions

### Defining the genome and reading a real marker catalogue

In [2]:
mutRate    = 0.0
numChr     = 2
nLoci      = 6096
chrLength  = [1.62;1.41]
numLoci    = [3339;2757]
mutationRate = 0.0
myData = readdlm("markerCatalogue4JuliaChrom1-2", ' ', Any, '\n', header=false)
mp1 = Float64.(myData[1,1:numLoci[1]])
mp2 = Float64.(myData[2,1:numLoci[2]])
mapPos = [mp1, mp2]

#this is not used just as a stub
genefreq1   = fill(0.5,numLoci[1])
genefreq2 = fill(0.5,numLoci[2])
geneFreq = [genefreq1, genefreq2]
#end stub

idx = rand(numLoci[1]).>0.995  # you want 0.5% to be QTL, i.e about 17 QTL
qtlIndex1 = collect(1:numLoci[1])[idx]
idx = rand(numLoci[2]).>0.995  # you want 0.5%% to be QTL, i.e about 14 QTL
qtlIndex2 = collect(1:numLoci[2])[idx]
qtlIndex = [qtlIndex1, qtlIndex2]
line = 0
numQTL = 0

for i in qtlIndex
    line +=1
    println("Number of QTL on chromosome $line: ", length(i))
    numQTL += 1
end

qtlEffect1 = randn(length(qtlIndex1))/sqrt(0.5*numQTL)
qtlEffect2 = randn(length(qtlIndex2))/sqrt(0.5*numQTL)
qtlEffect = [qtlEffect1, qtlEffect2];

Number of QTL on chromosome 1: 16
Number of QTL on chromosome 2: 10


### Building the genome
Passing  arrays for chrLength, numLoci and arrays of arrays for mapPos, geneFreq, qtlIndex, qtlEffect
i.e. using the most flexible version of build_genome

In [3]:
build_genome(numChr,chrLength,numLoci,geneFreq, mapPos, qtlIndex, qtlEffect, mutationRate)

### Sampling founder individuals from a file with 1326 genotyped individuals
Note that the genotype data for two chromosomes on the file is phased. You may have to use FImpute or some similar software for phasing. You may read less individuals form the data file than it contains, but not more.

In [4]:
popSizeFounder = 1326
basePop = sampleFounders(popSizeFounder, "reformattedMarkerDataChrom1-2");

Sampling 1326 animals into base population.


Splitting the founder individuals into tow cohorts of equal size

In [5]:
basePopMales = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0))
basePopMales.animalCohort = basePop.animalCohort[1:663];

In [6]:
basePopFemales = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0))
basePopFemales.animalCohort = basePop.animalCohort[664:end];

Although not used here, animals and cohorts belong to breeds, and the breed coposition can be set for cohorts and is inferered thereafter from the mating strategy

In [7]:
XSim.setBreedComp(basePop,[1.0])
XSim.setBreedComp(basePopMales,[1.0])
XSim.setBreedComp(basePopFemales,[1.0]);

Removing output files form a previous run, then creating 5'000 DH lines from an initial crossing of the two base populations

In [10]:
outputFileName = "JuliaPlants"
run(`\rm -f $outputFileName.ped`)
run(`\rm -f $outputFileName.phe`)
run(`\rm -f $outputFileName.brc`)
run(`\rm -f $outputFileName.gen`)

numOffspring = 5000
DHBaseLines1 = XSim.sampleDHLines(basePopMales, basePopFemales, numOffspring)
DHBaseLines2 = XSim.sampleDHLines(basePopMales, basePopFemales, numOffspring);

Lets look at the mean and variance of the genotypic values of our 5000 DH-lines in DHBaseline1 and DHBaseline2.  

In [11]:
var(getOurGenVals(DHBaseLines1))

8.545332099303963

In [15]:
var(getOurGenVals(DHBaseLines2))

8.163425372420138

In [13]:
mean(getOurGenVals(DHBaseLines1))

5.898931152544866

In [16]:
mean(getOurGenVals(DHBaseLines2))

5.83834164615054

Let us specify  the residual variance to simulate phenotypic values, note that getOurPhenVals() is actually setting all individuals phenotypic values in the cohort that is passed as an argument.

In [17]:
resVar = 5
XSim.setResidualVariance(resVar)

5

In [18]:
var(getOurPhenVals(DHBaseLines1,resVar))

13.291915329516172

In [19]:
var(getOurPhenVals(DHBaseLines2,resVar))

12.825604708501263

In [20]:
outputFileName

"JuliaPlants"

Now lets output the data to the files. Note that outputPedigree() is not only outputting the pedigree, but also the phenotypes and gentoypes, look at the files that got generated!

In [23]:
outputPedigree(DHBaseLines1,outputFileName);
outputPedigree(DHBaseLines2,outputFileName);

Sampling each 2500 male and female parents from DHBaseline1 at random and producing one offspring each. Sampling of parents is done with replacement. Note that sampleRan() is sampling 50% female and 50% male offspring, we are concatenating them into a single cohort. Note further, that we are using the same population to sample male and female parents from, so sex is ignored and we might even end up cloning an individual. Furthermore, after this step of random mating, our DH-Lines are not double haploids anymore. With the parameter numGen that is passed as an argument to sampleRan(), we are determining the number of generations of random mating. The variable 'gen' is used to store the actual numer of generation the simualtion currently is at.

In [24]:
DHLine1_H1 = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0)) 
DHLine1_H1m = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0)) 
DHLine1_H1f = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0))
numOffspring = 2500
numGen = 1
gen = 1
output = true
DHLine1_H1m, DHLine1_H1f, gen = sampleRan(numOffspring, numGen, DHBaseLines1, DHBaseLines1, fileName=outputFileName)
DHLine1_H1 = concatCohorts(DHLine1_H1m, DHLine1_H1f);

Generation     2: sampling  1250 males and  1250 females


Now we are repeating the same random mating for DHBaseline2

In [25]:
DHLine2_H1 = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0)) 
DHLine2_H1m = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0)) 
DHLine2_H1f = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0))
numOffspring = 2500
numGen = 1
gen = 1
output = true
DHLine2_H1m, DHLine2_H1f, gen = sampleRan(numOffspring, numGen, DHBaseLines2, DHBaseLines2, fileName=outputFileName)
DHLine2_H1 = concatCohorts(DHLine2_H1m, DHLine2_H1f);

Generation     2: sampling  1250 males and  1250 females


We know now how to generate double haploid lines and how to do random mating. Next let's do phenotypic selection. As before we are declaring the male and female cohorts to hold the offspring that are going to be generated. We will be generation 1000 offspring based on 100 phenotypically best male and female parents. Again we will be selecting the phenotypically best 100 parents from the same population that was generated in the previous step (DHLine1_H1).

In [26]:
DHLine1_H2 = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0)) 
DHLine1_H2m = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0)) 
DHLine1_H2f = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0))
numMaleParents = 100
numFemaleParents = 100
numOffspring = 1000
numGen = 1
DHLine1_H2m, DHLine1_H2f, gen = sampleSel(numOffspring, numMaleParents, numFemaleParents, numGen, 
                                    DHLine1_H1, DHLine1_H1, XSim.common.varRes, gen=gen,
                                    fileName=outputFileName, direction=1);
DHLine1_H2 = concatCohorts(DHLine1_H2m, DHLine1_H2f);

Generation     3: sampling   500 males and   500 females


As before, we are applying mass selection also to the other population in the same way as was done above for DHLine_H1 now for DHLine_H2

In [27]:
DHLine2_H2 = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0)) 
DHLine2_H2m = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0)) 
DHLine2_H2f = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0))
numMaleParents = 100
numFemaleParents = 100
numOffspring = 1000
numGen = 1
DHLine2_H2m, DHLine2_H2f, gen = sampleSel(numOffspring, numMaleParents, numFemaleParents, numGen, 
                                    DHLine2_H1, DHLine2_H1, XSim.common.varRes, gen=gen,
                                    fileName=outputFileName, direction=1);
DHLine2_H2 = concatCohorts(DHLine2_H2m, DHLine2_H2f);

Generation     4: sampling   500 males and   500 females


After random mating and phentoypic selection, let us apply selection based on BLUP, i.e. breeding values. For this purpose, we need to read the pedigree and phenotpyic files that were produced sampleRan() and sampleSel() as well as outputPedigree() above. First, let's read the phenotypic data into a dataframe.

In [28]:
using DataFrames
# BLUP selection
phenofile = outputFileName*".phe"
colNames = [:Animal;:y;:bv]
#dfPhen = CSV.read(phenofile,delim = ' ',header=false,names=colNames)
dfPhen = readtable(phenofile, separator=' ', header=false)
names!(dfPhen, colNames)
dfPhen[:1] = string.(dfPhen[:1])
dfPhen

┌ Warning: readtable is deprecated, use CSV.read from the CSV package instead
│   caller = ip:0x0
└ @ Core :-1


,Animal,y,bv
,String,Float64⍰,Float64⍰
1,11327,3.276,5.311
2,11328,-0.294,1.166
3,11329,4.294,5.629
4,11330,4.132,5.265
5,11331,3.19,4.993
6,11332,2.417,4.203
7,11333,0.968,4.606
8,11334,1.417,0.903
9,11335,-0.594,0.813


Now, we read the pedigree in to a dataframe, reformat it, write is out again such that it can finally be read by the get_pedigree() method. This should be fixed in future releases....

In [29]:
pedfile   = outputFileName*".ped"
dfPed = readtable(pedfile,separator = ' ', header = false)

,x1,x2,x3
,Int64⍰,Int64⍰,Int64⍰
1,11327,353,1183
2,11328,585,1173
3,11329,600,1174
4,11330,166,1248
5,11331,23,880
6,11332,459,1104
7,11333,115,699
8,11334,242,714
9,11335,176,902


In [30]:
using CSV
pedfile = outputFileName*".pedReformatted"
CSV.write(pedfile, dfPed, delim=';',header=["ind","sire" , "dam"])

"JuliaPlants.pedReformatted"

In [31]:
ped = XSim.get_pedigree(pedfile,separator=';');

coding pedigree... 100%|████████████████████████████████| Time: 0:00:00


Finished!


calculating inbreeding... 100%|█████████████████████████| Time: 0:00:00


After having read phenotypes and genotypes, we can now set the genetic variance that shall be used in BLUP, define the model for analysis, setup the mixed model equations and solve them. 

In [32]:
varGen = 5
mme = XSim.build_model("y = intercept + Animal",resVar)
XSim.set_random(mme,"Animal",ped,varGen)
out = XSim.solve(mme,dfPhen,solver="GaussSeidel",printout_frequency=40)

40 0.5597827780131326
80 0.10659597357821231
120 0.020298413644252583
160 0.0038653016874606776
200 0.0007360455553884282
240 0.00014016061396337343
280 2.6689921177808646e-5
320 5.0823970412049206e-6
360 9.678095066449212e-7


20830×2 Array{Any,2}:
 "1:intercept : intercept"   5.86002  
 "1:Animal : 395"            0.872509 
 "1:Animal : 1196"           0.0396781
 "1:Animal : 11381"          1.76631  
 "1:Animal : 400"            1.40622  
 "1:Animal : 1148"          -0.124251 
 "1:Animal : 13989"          0.664503 
 "1:Animal : 23986"          1.92027  
 "1:Animal : 505"           -1.19509  
 "1:Animal : 674"            1.85249  
 "1:Animal : 18565"         -1.62616  
 "1:Animal : 618"           -3.47699  
 "1:Animal : 1180"          -3.52481  
 ⋮                                    
 "1:Animal : 20561"         -1.72644  
 "1:Animal : 14760"         -0.868953 
 "1:Animal : 21556"          1.00329  
 "1:Animal : 26355"          1.66263  
 "1:Animal : 30462"          4.35022  
 "1:Animal : 26635"         -0.524683 
 "1:Animal : 28283"         -0.680487 
 "1:Animal : 12795"         -0.787815 
 "1:Animal : 22128"          0.44437  
 "1:Animal : 25062"          1.94814  
 "1:Animal : 22690"          0.506108 
 "1

Now all our indiviudals have estimated genotypic values, with the method putEBV() we are transferring them to the indiviudals that we would like to select to become the parents of the next generation. In our case, these would be the cohorts DHLine1_H2 and DHLine2_H2

In [33]:
# transfer BLUP-EBV to animals
XSim.putEBV(DHLine1_H2,ped,mme,out)
XSim.putEBV(DHLine2_H2,ped,mme,out)

In [34]:
DHLine1_H2_sorted = XSim.Cohort(Array{XSim.Animal}(undef,0),Array{Int64}(undef,0,0))
DHLine2_H2_sorted  = XSim.Cohort(Array{XSim.Animal}(undef,0),Array{Int64}(undef,0,0))
DHLine1_H3  = XSim.Cohort(Array{XSim.Animal}(undef,0),Array{Int64}(undef,0,0))
DHLine2_H3  = XSim.Cohort(Array{XSim.Animal}(undef,0),Array{Int64}(undef,0,0))
direction = 1.0
numMaleParents = 50
numFemaleParents = 50
numOffspring = 500
numGen = 1

1

After having declared empty cohorts for the offspring of the next generation, we are creating a y vector with the animals EBVs in the direction we want to select on (e.g. if low values arefavourable, direction should be -1.0). Then EBVs are sorted and the best number of parents are selected to be passed as parents to sampleChildren()

In [35]:
println("Generation $gen : sampling children by BLUP")
y = direction*[animal.ebv for animal in DHLine1_H2.animalCohort]
DHLine1_H2_sorted.animalCohort = DHLine1_H2.animalCohort[sortperm(y)][(end-numMaleParents+1):end]
y = direction*[animal.ebv for animal in DHLine2_H2.animalCohort]
DHLine2_H2_sorted.animalCohort = DHLine2_H2.animalCohort[sortperm(y)][(end-numFemaleParents+1):end]
DHLine1_H3 = XSim.sampleChildren(DHLine1_H2_sorted,DHLine1_H2_sorted,numOffspring)
DHLine2_H3 = XSim.sampleChildren(DHLine2_H2_sorted,DHLine2_H2_sorted,numOffspring)
outputPedigree(DHLine1_H3,outputFileName)
outputPedigree(DHLine2_H3,outputFileName)
gen+=1

Generation 4 : sampling children by BLUP


5

Finally, we are producing each 250 double haploid lines again, from the offspring of the BLUP selected parents 

In [36]:
println("Producing DHLines again....")
DHLine1_4  = XSim.Cohort(Array{XSim.Animal}(undef,0),Array{Int64}(undef,0,0))
DHLine2_4  = XSim.Cohort(Array{XSim.Animal}(undef,0),Array{Int64}(undef,0,0))

numOffspring = 250
DHLine1_4 = XSim.sampleDHLines(DHLine1_H3, DHLine1_H3, numOffspring)
DHLine2_4 = XSim.sampleDHLines(DHLine2_H3, DHLine2_H3, numOffspring);
     


Producing DHLines again....


In [37]:
outputPedigree(DHLine1_4,outputFileName);
outputPedigree(DHLine2_4,outputFileName);

The final step is crossing the two DHLines again to produce hybrids....

In [38]:
println("Producing hybrids now....")
H1 = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0)) 
H1m = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0)) 
H1f = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0))
numOffspring = 250
numGen = 1
gen = 1
output = true
H1m, H1f, gen = sampleRan(numOffspring, numGen, DHLine1_4, DHLine2_4, fileName=outputFileName)
H1 = concatCohorts(DHLine2_H1m, DHLine2_H1f);


Producing hybrids now....
Generation     2: sampling   125 males and   125 females


Of course the 'breeding program' outlined here is not reflecting a real situation but rahter shows you the tools XSim has to do simulations. You should be able to modify the above code to better reflect a real situation. Also you may modify XSim's methodsto better reflect the situation you need to cope with. Note that all data simulated is written to the files with the name <outputFileName> that you declared above. so You should be able to look at selection response, genetic trend across generation etc.